In [ ]:
%pip install beautifulsoup4 azure-storage-blob
%pip install boto3
%pip install tabula-py 
%pip install tabulate
%pip install PyPDF2
%pip install pdfplumber

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import requests
import certifi

# Step 1: Fetch the data from the API endpoint
api_url = "https://data.cityofnewyork.us/resource/gra9-xbjk.csv"
csv_file_name = "restaurant_grades.csv"  # Name of the file to save locally and upload

try:
    # Fetch the data using requests with SSL verification
    response = requests.get(api_url, verify=certifi.where())
    response.raise_for_status()  # Raise an error for bad status codes

    # Convert the response content to a DataFrame
    data = StringIO(response.text)
    df = pd.read_csv(data)
    print("Data fetched successfully!")
    print(df.head())  # Display the first few rows of the DataFrame

    # Save the DataFrame to a local CSV file
    df.to_csv(csv_file_name, index=False)
    print(f"File '{csv_file_name}' saved locally.")
except Exception as e:
    print(f"An error occurred while fetching data: {e}")

# Step 2: Upload the file to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=strestaurantgrade;AccountKey=MTFwd/viTQS/J3dgVpgZ+8DVkAWeOa+a0yfMf/QkuxV63bWao99w7cvM3Hf9+6ayQeA0lkWH6Vis+AStHFgr3Q==;EndpointSuffix=core.windows.net"  # Replace with your Azure Storage connection string
container_name = "restaurantdata"        
blob_name = csv_file_name                          # Use the same name for the blob

try:
    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get a BlobClient for the container and blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Upload the CSV file to Azure Blob Storage
    with open(csv_file_name, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

    print(f"File '{csv_file_name}' uploaded to Azure Blob Storage successfully!")
except Exception as e:
    print(f"An error occurred while uploading to Azure Blob Storage: {e}")